# Preprocessing Menggunakan Spacy

## memakai NLTK

In [1]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [2]:
def word_tokenize_wrapper(text):
 return word_tokenize(text)

In [3]:
teks_nltk = word_tokenize_wrapper('Teknologi telah mengubah dunia secara fundamental, memengaruhi segala aspek kehidupan manusia dari komunikasi hingga industri, dengan terobosan seperti kecerdasan buatan, komputasi kuantum, dan Internet of Things. Meskipun membawa manfaat besar, kita juga dihadapkan pada tantangan seperti privasi data dan kesenjangan digital. Oleh karena itu, penting untuk terus mengembangkan teknologi secara bertanggung jawab demi meningkatkan kesejahteraan semua orang.')

In [4]:
teks_nltk

['Teknologi',
 'telah',
 'mengubah',
 'dunia',
 'secara',
 'fundamental',
 ',',
 'memengaruhi',
 'segala',
 'aspek',
 'kehidupan',
 'manusia',
 'dari',
 'komunikasi',
 'hingga',
 'industri',
 ',',
 'dengan',
 'terobosan',
 'seperti',
 'kecerdasan',
 'buatan',
 ',',
 'komputasi',
 'kuantum',
 ',',
 'dan',
 'Internet',
 'of',
 'Things',
 '.',
 'Meskipun',
 'membawa',
 'manfaat',
 'besar',
 ',',
 'kita',
 'juga',
 'dihadapkan',
 'pada',
 'tantangan',
 'seperti',
 'privasi',
 'data',
 'dan',
 'kesenjangan',
 'digital',
 '.',
 'Oleh',
 'karena',
 'itu',
 ',',
 'penting',
 'untuk',
 'terus',
 'mengembangkan',
 'teknologi',
 'secara',
 'bertanggung',
 'jawab',
 'demi',
 'meningkatkan',
 'kesejahteraan',
 'semua',
 'orang',
 '.']

## memakai spacy

In [5]:
!pip install spacy

In [6]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'
Teks = nlp('Teknologi telah mengubah dunia secara fundamental, memengaruhi segala aspek kehidupan manusia dari komunikasi hingga industri, dengan terobosan seperti kecerdasan buatan, komputasi kuantum, dan Internet of Things. Meskipun membawa manfaat besar, kita juga dihadapkan pada tantangan seperti privasi data dan kesenjangan digital. Oleh karena itu, penting untuk terus mengembangkan teknologi secara bertanggung jawab demi meningkatkan kesejahteraan semua orang.')

In [7]:
Token_kata = [token.text for token in Teks]
Token_kata

['Teknologi',
 'telah',
 'mengubah',
 'dunia',
 'secara',
 'fundamental',
 ',',
 'memengaruhi',
 'segala',
 'aspek',
 'kehidupan',
 'manusia',
 'dari',
 'komunikasi',
 'hingga',
 'industri',
 ',',
 'dengan',
 'terobosan',
 'seperti',
 'kecerdasan',
 'buatan',
 ',',
 'komputasi',
 'kuantum',
 ',',
 'dan',
 'Internet',
 'of',
 'Things',
 '.',
 'Meskipun',
 'membawa',
 'manfaat',
 'besar',
 ',',
 'kita',
 'juga',
 'dihadapkan',
 'pada',
 'tantangan',
 'seperti',
 'privasi',
 'data',
 'dan',
 'kesenjangan',
 'digital',
 '.',
 'Oleh',
 'karena',
 'itu',
 ',',
 'penting',
 'untuk',
 'terus',
 'mengembangkan',
 'teknologi',
 'secara',
 'bertanggung',
 'jawab',
 'demi',
 'meningkatkan',
 'kesejahteraan',
 'semua',
 'orang',
 '.']

# load library

In [8]:
import re
import string
import time
from copy import deepcopy

# load dataset

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('review_dana_labelled.csv')

In [11]:
df.head()

userName  score                   at  \
0     Elisya Kasni      5  2024-02-15 11:24:56   
1       Rusman Man      2  2024-02-15 11:24:03   
2     Qiliw Sadega      1  2024-02-15 11:23:34   
3  Kijutjrv2 Kijut      3  2024-02-15 11:22:46   
4     Fifi Alfiyah      1  2024-02-15 11:21:34   

                                             content  sentimen  
0                                              Bagus  POSITIVE  
1                             Dana mmg keren mantap.  POSITIVE  
2  Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE  
3  Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE  
4  Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE

In [12]:
df = df.drop(['userName','score','at'], axis=1)
df

content  sentimen
0                                                  Bagus  POSITIVE
1                                 Dana mmg keren mantap.  POSITIVE
2      Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE
3      Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE
4      Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE
...                                                  ...       ...
49995                                              Bagus  POSITIVE
49996                            error, tidak bisa masuk  NEGATIVE
49997                                     Cukup membantu  POSITIVE
49998  tolong pihak dana knp saldo saya hilang,ada yg...   NEUTRAL
49999                                            good. 👌  NEGATIVE

[50000 rows x 2 columns]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   50000 non-null  object
 1   sentimen  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


# Preprocessing dataset

In [14]:
#!pip install ekphrasis

In [15]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\RESTU\AppData\Local\Programs\Python\Python312\Lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...


C:\Users\RESTU\AppData\Local\Programs\Python\Python312\Lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [16]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [ ]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [18]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## lakukan pembersihan dengan memanggil fungsi yang didefinisikan diatas

In [19]:
i = 0
final_string = []
s = ""
for text in df['content'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    #proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [20]:
df["step01"] = final_string

In [21]:
df.head(10)

content  sentimen  \
0                                              Bagus  POSITIVE   
1                             Dana mmg keren mantap.  POSITIVE   
2  Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE   
3  Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE   
4  Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE   
5                                              mayan  POSITIVE   
6  Udah gua hapus dana ya. ilang ya udah 1 juta l...  NEGATIVE   
7                                               baik  POSITIVE   
8  TOLONG UNTUK SISTEM KEAMANAN DI PERBAIKI. KALA...   NEUTRAL   
9                               mempermudah transfer  POSITIVE   

                                              step01  
0                                              bagus  
1                              dana mmg keren mantap  
2  saya ngajuin upgrade dana premium krna ktp say...  
3  kocak mana diskon nya ml malah eror segala kag...  
4  saldo hilang karena no lama hilang ganti no sa...  
5                                              mayan  
6  udah gua hapus dana ya ilang ya udah   juta le...  
7                                               baik  
8  tolong untuk sistem keamanan di perbaiki kalau...  
9                               mempermudah transfer

## hapus data kosong

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   50000 non-null  object
 1   sentimen  50000 non-null  object
 2   step01    50000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [23]:
df_hapus = df[~df['step01'].str.contains(" ")]
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14563 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   14563 non-null  object
 1   sentimen  14563 non-null  object
 2   step01    14563 non-null  object
dtypes: object(3)
memory usage: 455.1+ KB


In [24]:
df_hapus.head(10)

content  sentimen       step01
0         Bagus  POSITIVE        bagus
5         mayan  POSITIVE        mayan
7          baik  POSITIVE         baik
14      ookeeee  POSITIVE      ookeeee
20         fast  POSITIVE         fast
27        Cukup  POSITIVE        cukup
29  mantapppppp  POSITIVE  mantapppppp
31        mntap  POSITIVE        mntap
39       mantap  POSITIVE       mantap
46       Mantap  POSITIVE       mantap

In [25]:
df_new = df[~df.isin(df_hapus)].dropna()
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35437 entries, 1 to 49998
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   35437 non-null  object
 1   sentimen  35437 non-null  object
 2   step01    35437 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [26]:
df_new

content  sentimen  \
1                                 Dana mmg keren mantap.  POSITIVE   
2      Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE   
3      Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE   
4      Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE   
6      Udah gua hapus dana ya. ilang ya udah 1 juta l...  NEGATIVE   
...                                                  ...       ...   
49993  Amanah, biasanya kerja juga pake apk dana buat...   NEUTRAL   
49994  Kak tolong perbaiki dana saya, Karena tidak mu...   NEUTRAL   
49996                            error, tidak bisa masuk  NEGATIVE   
49997                                     Cukup membantu  POSITIVE   
49998  tolong pihak dana knp saldo saya hilang,ada yg...   NEUTRAL   

                                                  step01  
1                                  dana mmg keren mantap  
2      saya ngajuin upgrade dana premium krna ktp say...  
3      kocak mana diskon nya ml malah eror segala kag...  
4      saldo hilang karena no lama hilang ganti no sa...  
6      udah gua hapus dana ya ilang ya udah   juta le...  
...                                                  ...  
49993  amanah biasanya kerja juga pake apk dana buat ...  
49994  kak tolong perbaiki dana saya karena tidak mun...  
49996                             error tidak bisa masuk  
49997                                     cukup membantu  
49998  tolong pihak dana knp saldo saya hilang ada yg...  

[35437 rows x 3 columns]

## normalisasi kata slang menjadi kata baku

In [27]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [28]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [29]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [30]:
df_new.head(10)

content  sentimen  \
1                              Dana mmg keren mantap.  POSITIVE   
2   Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE   
3   Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE   
4   Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE   
6   Udah gua hapus dana ya. ilang ya udah 1 juta l...  NEGATIVE   
8   TOLONG UNTUK SISTEM KEAMANAN DI PERBAIKI. KALA...   NEUTRAL   
9                                mempermudah transfer  POSITIVE   
10  baru yobain semoga aman sentosa tidak pembobol...   NEUTRAL   
11                                kok gx bisa keridit  NEGATIVE   
12                                        sangat puas  POSITIVE   

                                               step01  \
1                               dana mmg keren mantap   
2   saya ngajuin upgrade dana premium krna ktp say...   
3   kocak mana diskon nya ml malah eror segala kag...   
4   saldo hilang karena no lama hilang ganti no sa...   
6   udah gua hapus dana ya ilang ya udah   juta le...   
8   tolong untuk sistem keamanan di perbaiki kalau...   
9                                mempermudah transfer   
10  baru yobain semoga aman sentosa tidak pembobol...   
11                                kok gx bisa keridit   
12                                        sangat puas   

                                               tokens  
1                          [dana, mmg, keren, mantap]  
2   [saya, ngajuin, upgrade, dana, premium, krna, ...  
3   [kocak, mana, diskon, nya, ml, malah, eror, se...  
4   [saldo, hilang, karena, no, lama, hilang, gant...  
6   [udah, gua, hapus, dana, ya, ilang, ya, udah, ...  
8   [tolong, untuk, sistem, keamanan, di, perbaiki...  
9                             [mempermudah, transfer]  
10  [baru, yobain, semoga, aman, sentosa, tidak, p...  
11                           [kok, gx, bisa, keridit]  
12                                     [sangat, puas]

In [31]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [32]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

C:\Users\RESTU\AppData\Local\Temp\ipykernel_12044\2408214049.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\RESTU\AppData\Local\Temp\ipykernel_12044\2408214049.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [33]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [34]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [35]:
df_new["step02"] = final_string_tokens

In [36]:
df_new.head(10)

content  sentimen  \
1                              Dana mmg keren mantap.  POSITIVE   
2   Saya ngajuin upgrade dana premium krna ktp say...  NEGATIVE   
3   Kocak mana diskon nya ml malah eror segala kag...  NEGATIVE   
4   Saldo hilang karena no lama Hilang ganti no sa...  NEGATIVE   
6   Udah gua hapus dana ya. ilang ya udah 1 juta l...  NEGATIVE   
8   TOLONG UNTUK SISTEM KEAMANAN DI PERBAIKI. KALA...   NEUTRAL   
9                                mempermudah transfer  POSITIVE   
10  baru yobain semoga aman sentosa tidak pembobol...   NEUTRAL   
11                                kok gx bisa keridit  NEGATIVE   
12                                        sangat puas  POSITIVE   

                                               step01  \
1                               dana mmg keren mantap   
2   saya ngajuin upgrade dana premium krna ktp say...   
3   kocak mana diskon nya ml malah eror segala kag...   
4   saldo hilang karena no lama hilang ganti no sa...   
6   udah gua hapus dana ya ilang ya udah   juta le...   
8   tolong untuk sistem keamanan di perbaiki kalau...   
9                                mempermudah transfer   
10  baru yobain semoga aman sentosa tidak pembobol...   
11                                kok gx bisa keridit   
12                                        sangat puas   

                                               tokens  \
1                          [dana, mmg, keren, mantap]   
2   [saya, ngajuin, upgrade, dana, premium, krna, ...   
3   [kocak, mana, diskon, nya, ml, malah, eror, se...   
4   [saldo, hilang, karena, no, lama, hilang, gant...   
6   [udah, gua, hapus, dana, ya, ilang, ya, udah, ...   
8   [tolong, untuk, sistem, keamanan, di, perbaiki...   
9                             [mempermudah, transfer]   
10  [baru, yobain, semoga, aman, sentosa, tidak, p...   
11                           [kok, gx, bisa, keridit]   
12                                     [sangat, puas]   

                                         final_tokens  \
1                          [dana, mmg, keren, mantap]   
2   [saya, ngajuin, upgrade, dana, premium, krna, ...   
3   [kocak, mana, diskon, nya, ml, malah, eror, se...   
4   [saldo, hilang, karena, no, lama, hilang, gant...   
6   [udah, gua, hapus, dana, ya, ilang, ya, udah, ...   
8   [tolong, untuk, sistem, keamanan, di, perbaiki...   
9                             [mempermudah, transfer]   
10  [baru, yobain, semoga, aman, sentosa, tidak, p...   
11                           [kok, gx, bisa, keridit]   
12                                     [sangat, puas]   

                                               step02  
1                               dana mmg keren mantap  
2   saya ngajuin upgrade dana premium krna ktp say...  
3   kocak mana diskon nya ml malah eror segala kag...  
4   saldo hilang karena no lama hilang ganti no sa...  
6   udah gua hapus dana ya ilang ya udah juta lebi...  
8   tolong untuk sistem keamanan di perbaiki kalau...  
9                                mempermudah transfer  
10  baru yobain semoga aman sentosa tidak pembobol...  
11                                kok gx bisa keridit  
12                                        sangat puas

# Simpan

In [37]:
df.to_csv('clean_dataset2.csv',sep=";")